In [28]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn, optim

# Basic pytorch setup

In [29]:
# Connect torch to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [30]:
# Load data
df = pd.read_csv('data/dataset_india_big.csv')

# Pick a 

# Convert to numpy arrays
nn_input = []
frequencies = []

counter = 1
min_length = float('inf')

# Split each frequency into a list of data for input to the neural network
for frequency in df['Frequency'].unique():
    print(f'Frequency {counter} is "{frequency}"')
    frequencies.append(frequency)
    counter += 1
    
    frequency_data = df[df['Frequency'] == frequency]['Signal Strength'].values
    nn_input.append(frequency_data)
    
    if len(frequency_data) < min_length:
        min_length = len(frequency_data)
        
# Truncate all data to the length of the shortest frequency
for i in range(len(nn_input)):
    nn_input[i] = nn_input[i][:min_length]

# Place data in numpy array
nn_input = np.array(nn_input)

nn_input = nn_input.T

Frequency 1 is "120000000"
Frequency 2 is "160000000"
Frequency 3 is "90000000"
Frequency 4 is "70000000"
Frequency 5 is "100000000"
Frequency 6 is "140000000"


# Description of input variables

- `input_size` is the amount of frequencies that have been measured
- `hidden_size` is the amount of neurons in the hidden layer
- `num_layers` is the amount of hidden layers that perform changes to get the correct prediction (chosen based on https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw)
- `output_size` is the amount of frequencies that are predicted, these are equal to the input since we want to see from the full prediction which frequency most likely has the lowest interference
- `seq_length` is the amount of history that gets taken into account to make the new prediction
- `num_epochs` is the amount of training rounds
- `learning_rate` is the rate at which the weights of the hidden layers are updated to improve prediction results (cannot be too high because it might overshoot)


In [31]:
data = torch.FloatTensor(nn_input)

print(data.shape)

# Neural network input
input_size = data.shape[1]
hidden_size = input_size
num_layers = 1
output_size = input_size
seq_length = 12
num_epochs = 100
learning_rate = 0.001

torch.Size([27051, 6])


In [32]:
# Create sequences and labels for training
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length]
        sequences.append(seq)
        labels.append(label)
    return torch.stack(sequences), torch.stack(labels)

# Parameters
sequences, labels = create_sequences(data, seq_length)

sequences = sequences.to(device)
labels = labels.to(device)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)  # Hidden state
        c_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)  # Cell state
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])  # Fully connected on the last output
        return out

# Initialize the model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
# Initialize loss
loss = float('inf')

epoch = 0

# Training loop
while loss > 1000:
    epoch += 1
    
    outputs = model(sequences)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        sys.stdout.write(f'\rEpoch {epoch + 1}, Loss: {loss.item():.4f}            ')
        sys.stdout.flush()


Epoch 6200, Loss: 1004.1107            

In [36]:
# Test prediction for the next value in the sequence
with torch.no_grad():
    test_input = data[-seq_length:].unsqueeze(0).to(device)  # Use the last 3 values
    predicted_frequencies = model(test_input)
    
    results = predicted_frequencies[0].cpu().numpy()
    
    print(f'Predicted next value:')
    
    for i, result in enumerate(results):
        if i == len(results) - 1:
            sys.stdout.write(f'Frequency_{i + 1}: {result:.4f} dB\n')
        else:
            sys.stdout.write(f'Frequency_{i + 1}: {result:.4f} dB | ')
    
    # Post-prediction analysis
    # Find the frequency that is predicted to be least occupied (based on some custom metric)
    # Example: Choose frequency with the lowest predicted value (or you can apply a more specific condition)
    best_frequency_index = torch.argmax(predicted_frequencies[-1, :])  # Last time step prediction
    print(f"\nThe next likely available frequency is: {frequencies[best_frequency_index]}")

Predicted next value:
Frequency_1: -37.8916 dB | Frequency_2: -37.3596 dB | Frequency_3: -37.6615 dB | Frequency_4: -37.8767 dB | Frequency_5: -37.4292 dB | Frequency_6: -38.3638 dB

The next likely available frequency is: 160000000
